# 3D Droplet Oscillation (Part 2, Postprocessing)

Results published: hopefully at some point!

This is part of the BoSSS-long-term validation test suite, which consists of several computationally expensive test-cases (runtime in the order of days), which are performed on a regular basis in order to validate the physical correctness of BoSSS simulations.

### Preliminaries

This example can be found in the source code repository as as `Droplet3D-Postprocessing.ipynb`. 
One can directly load this into Jupyter to interactively work with the following code examples.

Note: First, BoSSS has to be loaded into the Jupyter kernel. Note:
In the following line, the reference to `BoSSSpad.dll` is required. 
One must either set `#r "BoSSSpad.dll"` to something which is appropirate for the current computer
(e.g. `C:\Program Files (x86)\FDY\BoSSS\bin\Release\net5.0\BoSSSpad.dll` if working with the binary distribution), 
or, if one is working with the source code, one must compile `BoSSSpad`
and put it side-by-side to this worksheet file 
(from the original location in the repository, one can use the scripts `getbossspad.sh`, resp. `getbossspad.bat`).


In [1]:
//#r "../../src/L4-application/BoSSSpad/bin/Release/net5.0/BoSSSpad.dll"
//#r "../../src/L4-application/BoSSSpad/bin/Debug/net5.0/BoSSSpad.dll"
#r "BoSSSpad.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Foundation;
using BoSSS.Foundation.XDG;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.AdvancedSolvers;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
Init();

The below script needs to be able to find the current output cell; this is an easy method to get it.

## Initialization tasks

Loading the `XNSE_Solver` and additional namespace:

In [2]:
using BoSSS.Application.XNSE_Solver;
using BoSSS.Application.XNSE_Solver.PhysicalBasedTestcases;
using BoSSS.Solution.NSECommon;
using BoSSS.Solution.LevelSetTools.SolverWithLevelSetUpdater;
using NUnit.Framework;
using BoSSS.Application.XNSE_Solver.Logging;

Initialization of the Workflow management; there `OscillatingDroplet3D` is the project name which is used name all computations (aka. sessions):

In [3]:
BoSSSshell.WorkflowMgm.Init("OscillatingDroplet3D");

Project name is set to 'OscillatingDroplet3D'.
Opening existing database '\\fdygitrunner\ValidationTests\OscillatingDroplet3D'.


In [4]:
//add database manually:
//OpenOrCreateDatabase(@"\\fdygitrunner\ValidationTests\OscillatingDroplet3D");

In [5]:
wmg.Sessions

#0: OscillatingDroplet3D	J432k3_arm0_case2_Oh0.1*	12/10/2021 11:43:26	e6311bec...
#1: OscillatingDroplet3D	J432k3_arm0_case2_Oh0.1*	12/10/2021 10:18:08	02e64152...
#2: OscillatingDroplet3D	J432k3_arm0_case2_Oh0.1*	12/10/2021 08:35:28	a494d808...
#3: OscillatingDroplet3D	J432k3_arm0_case2_Oh0.1*	12/10/2021 08:19:12	d696607a...
#4: OscillatingDroplet3D	J432k3_arm0_case2_Oh0.1*	12/10/2021 06:57:44	09ff19b0...
#5: OscillatingDroplet3D	J432k3_arm0_case2_Oh0.1*	12/10/2021 06:54:17	cf8b45c4...
#6: OscillatingDroplet3D	J432k3_arm0_case2_Oh0.1*	12/10/2021 06:50:27	27b1b041...
#7: OscillatingDroplet3D	J432k3_arm0_case2_Oh0.1*	12/10/2021 06:45:20	02b1efa6...
#8: OscillatingDroplet3D	J432k3_arm0_case2_Oh0.1*	12/10/2021 06:26:46	721751d8...
#9: OscillatingDroplet3D	J432k3_arm0_case2_Oh0.1*	12/10/2021 06:15:04	a5ef48c3...
#10: OscillatingDroplet3D	J432k3_arm0_case2_Oh0.1*	12/10/2021 05:39:29	a08da869...
#11: OscillatingDroplet3D	J432k3_arm0_case2_Oh0.1*	12/09/2021 16:33:28	6a3699ae...
#12: Oscillati

In [6]:
using System.IO;

In [7]:
/*
// temporary fix for a bug in the output file "SphericalHarmonics.txt"
// (missing separator/tab between fist ans second column)
void FileSanitizer(string _TextFile) {
Console.WriteLine("sanitizing: " + _TextFile);
var TextFile2 = Path.Combine(Path.GetDirectoryName(_TextFile), Path.GetFileNameWithoutExtension(_TextFile) + "-Copy.txt");
File.Copy(_TextFile,TextFile2, true);
var TextFile3 = Path.Combine(Path.GetDirectoryName(_TextFile), Path.GetFileNameWithoutExtension(_TextFile) + "-Sanitized.txt");
using (StreamWriter wrt = new StreamWriter(TextFile3)) {
using (StreamReader reader = new StreamReader(new FileStream(TextFile2, FileMode.Open, FileAccess.Read, FileShare.Read))) {
  int cnt = 0;
  for(string l = reader.ReadLine(); l != null; l = reader.ReadLine()) {
     if(cnt >= 1) {
       string[] parts = l.Split("\t");
       //Console.WriteLine(parts[0] + "    " + parts.Length);
       string P0 = parts[0];
       int L = P0.Length;
       
       string pp1, pp2;
       int exp = P0.IndexOf("E", 0, 5);
       //exp = -1;
       if(exp >= 0) {
          int ppl = P0[exp + 1] == '-' ? 4 : 3;
          pp1 = P0.Substring(0,exp + ppl);
          pp2 = P0.Substring(exp + ppl);
          
       } else {
          int minus = P0.IndexOf("-");
          if(minus >= 0 && P0[minus - 1] == 'E')
             minus = -1;
          if(minus >= 0) {
              pp1 = P0.Substring(0,minus);
              pp2 = P0.Substring(minus);
          } else {
              int comma = P0.IndexOf('.', 2, L - 2);
              pp1 = P0.Substring(0,comma - 1);
              pp2 = P0.Substring(comma - 1);
          }
       }
       try {
          double.Parse(pp1);
       } catch(Exception) {
          Console.Error.WriteLine("line " + (cnt + 1) + " cannot parse 1: " + pp1 + "  from " + P0);
       }
       try {
            double.Parse(pp2);
       } catch(Exception) {
          Console.Error.WriteLine("line " + (cnt + 1) + " cannot parse 2: " + pp2 + "  from " + P0);
       }
       wrt.Write(pp1 + "\t" + pp2);

       
       for(int i = 1; i < parts.Length; i++) {
            wrt.Write("\t");
            wrt.Write(parts[i]);
       }
       wrt.WriteLine();
       
     } else {
       wrt.WriteLine(l);
     }
     cnt++;
  }
}
}

}
*/

In [9]:
/*
foreach(var s in wmg.Sessions) {
    string dir = DatabaseDriver.GetSessionDirectory(s);
    string file = Path.Combine(dir, "SphericalHarmonics.txt");
    FileSanitizer(file);
}
*/

In [10]:
/*var plot = new Plot2Ddata();
var allColors = Enum.GetValues(typeof(LineColors)).Cast<LineColors>().ToArray();
var time = tab["time"];
int cnt = -1;
foreach(var column in tab) {
   cnt++;
   var fmt = new PlotFormat();
   fmt.Style = Styles.Lines; 
   fmt.LineColor = allColors[cnt%allColors.Length];
   if(column.Key == "time")
      continue;
   plot.AddDataGroup(column.Key, time, column.Value, fmt);
   
   plot.Title = S1.Name;
}*/

In [11]:
//plot.PlotNow()

In [12]:
wmg.Sessions

#0: OscillatingDroplet3D	J432k3_arm0_case2_Oh0.1*	12/10/2021 11:43:26	e6311bec...
#1: OscillatingDroplet3D	J432k3_arm0_case2_Oh0.1*	12/10/2021 10:18:08	02e64152...
#2: OscillatingDroplet3D	J432k3_arm0_case2_Oh0.1*	12/10/2021 08:35:28	a494d808...
#3: OscillatingDroplet3D	J432k3_arm0_case2_Oh0.1*	12/10/2021 08:19:12	d696607a...
#4: OscillatingDroplet3D	J432k3_arm0_case2_Oh0.1*	12/10/2021 06:57:44	09ff19b0...
#5: OscillatingDroplet3D	J432k3_arm0_case2_Oh0.1*	12/10/2021 06:54:17	cf8b45c4...
#6: OscillatingDroplet3D	J432k3_arm0_case2_Oh0.1*	12/10/2021 06:50:27	27b1b041...
#7: OscillatingDroplet3D	J432k3_arm0_case2_Oh0.1*	12/10/2021 06:45:20	02b1efa6...
#8: OscillatingDroplet3D	J432k3_arm0_case2_Oh0.1*	12/10/2021 06:26:46	721751d8...
#9: OscillatingDroplet3D	J432k3_arm0_case2_Oh0.1*	12/10/2021 06:15:04	a5ef48c3...
#10: OscillatingDroplet3D	J432k3_arm0_case2_Oh0.1*	12/10/2021 05:39:29	a08da869...
#11: OscillatingDroplet3D	J432k3_arm0_case2_Oh0.1*	12/09/2021 16:33:28	6a3699ae...
#12: Oscillati

In [17]:
//wmg.Sessions.Where(si => si.Name.Contains("case2")).ForEach(si => si.Delete(true));

In [27]:
//wmg.Sessions.Single(si => si.Name.Contains("case3")).Export().WithSupersampling(2).Do()

Starting export process... Data will be written to the directory: C:\Users\jenkinsci\AppData\Local\BoSSS\plots\sessions\OscillatingDroplet3D__J432k3_arm0_case3_Oh0.1__2abc0052-d91e-4564-b7e8-87c343eef063


C:\Users\jenkinsci\AppData\Local\BoSSS\plots\sessions\OscillatingDroplet3D__J432k3_arm0_case3_Oh0.1__2abc0052-d91e-4564-b7e8-87c343eef063

In [30]:
//File.WriteAllText("case3-restart.obj", wmg.Sessions.Single(si => si.Name.Contains("case3")).CreateRestartControl().Serialize());

In [26]:
foreach(var s in wmg.Sessions) {
    Console.WriteLine(s.Name);
    Console.WriteLine(s.DeployPath);
}

J432k3_arm0_case2_Oh0.1
\\fdygitrunner\ValidationTests\OscillatingDroplet3D-XNSE_Solver2021Dec09_143415
J432k3_arm0_case2_Oh0.1
\\fdygitrunner\ValidationTests\OscillatingDroplet3D-XNSE_Solver2021Dec09_143415
J432k3_arm0_case5_Oh0.56
\\fdygitrunner\ValidationTests\OscillatingDroplet3D-XNSE_Solver2021Dec09_143549
J432k3_arm0_case4_Oh0.1
\\fdygitrunner\ValidationTests\OscillatingDroplet3D-XNSE_Solver2021Dec09_143524
J432k3_arm0_case3_Oh0.1
\\fdygitrunner\ValidationTests\OscillatingDroplet3D-XNSE_Solver2021Dec09_143444
J432k3_arm0_case2_Oh0.1
\\fdygitrunner\ValidationTests\OscillatingDroplet3D-XNSE_Solver2021Dec09_143415
J432k3_arm0_case1_Oh0.1
\\fdygitrunner\ValidationTests\OscillatingDroplet3D-XNSE_Solver2021Dec09_143358


In [ ]:
string[] modes = new string[]{ "mode2", "mode3", "mode4"};
string[] aPis = new string[] { "aP0", "aP1", "aP2" };
string[] amrS = new string[] { "arm0", "arm1" };

In [ ]:
Plot2Ddata[,] PlotTable = new Plot2Ddata[3,3];
for(int iCol = 0; iCol < 3; iCol++) {
for(int iRow = 0; iRow < 3; iRow++) {
for(int iarm = 0; iarm < 2; iarm++) {
    string _mode = modes[iRow];
    string _aP = aPis[iCol];
    string _amr = amrS[iarm];

    ISessionInfo SI = wmg.Sessions.Single(sess => sess.Name.Contains(_mode) 
                                               && sess.Name.Contains(_aP) 
                                               && sess.Name.Contains(_amr));
    Console.WriteLine(SI.Name);

    var tab = SI.ReadTabulatedTextFileAsDoubles("SphericalHarmonics-Sanitized.txt", '\t');
    
    var plot = new Plot2Ddata();
    var allColors = Enum.GetValues(typeof(LineColors)).Cast<LineColors>().ToArray();
    var time = tab["time"];
    int cnt = -1;
    foreach(var column in tab) {
       cnt++;
       var fmt = new PlotFormat();
       fmt.Style = Styles.Lines; 
       fmt.LineColor = allColors[cnt%allColors.Length];
       if(column.Key == "time")
          continue;
       if(iarm > 0)
          fmt.DashType = DashTypes.Solid;
       else 
          fmt.DashType = DashTypes.Dashed;
       string name = column.Key;
       if(iarm > 0)
           name = name + "-amr";
       plot.AddDataGroup(name, time, column.Value, fmt);
   }
   
   plot.ShowLegend = iCol == 2 && iRow == 2;
   
   plot.Title = SI.Name;
   
   if(iarm == 0) {
       PlotTable[iRow,iCol] = plot;
   } else {
       PlotTable[iRow,iCol] = PlotTable[iRow,iCol].Merge(plot);
   }
}
PlotTable[iRow,iCol].ShowLegend = iCol == 2 && iRow == 2;
if(iRow == 0)
   PlotTable[iRow,iCol].Title = aPis[iCol];
if(iCol == 0)
   PlotTable[iRow,iCol].Ylabel = modes[iRow];
}
}


In [ ]:
var gp = PlotTable.ToGnuplot();
gp.PlotSVG(xRes:1800,yRes:1500)

In [ ]:
var cl = gp.PlotCairolatex(xSize:32,ySize:18);
cl.WriteMinimalCompileableExample("latex\\plot.tex");

In [ ]:
foreach(var S in wmg.Sessions) {
    //var EI = S.Export().WithSupersampling(1).Do();
}